## Necessary functions

In [2]:
# import pandas as pd
# trades = pd.read_parquet('trades_2023_09_28.parquet', engine='pyarrow')
# rates = pd.read_parquet('rates_2023_09_28_2.parquet', engine='pyarrow')
# ccy_details = pd.read_parquet('ccy_details.parquet', engine='pyarrow')

In [6]:
def zeus_execution(sql_qr):
    with zeus() as cur:
        df = execute_sql(cur,sql_qr,None)
    return df

query = '''
select *
from trading.conventional_fx_pairs'''

conventional_fx_pairs = zeus_execution(query).set_index('currency')

[2023-12-04 12:27:06.534380] Connected successfully.


In [52]:
def calculate_delta(row, base = None, counter = None, amount = None):

    if base is None:
        base = row['base_ccy']
    if counter is None:
        counter = row['counter_ccy']
    if amount is None:
        amount = row['total_base_amount']
    pair = base + counter

    # This filters out USDUSD and BSCBSC
    # Where BSC = balance sheet currency
    if base == counter:
        return

    def get_base_counter_rate():
        #by default
        usd_base = row['rateUSDBASE']
        usd_counter = row['rateUSDCOUNTER']
        #rewrite if in recursion and base is not original base
        if base != row['base_ccy']:
            if base == row['counter_ccy']:
                usd_base = row['rateUSDCOUNTER']
            else:
                if base == 'USD':
                    usd_base = 1
                else:
                    base_colname = 'rateUSD' + base
                    usd_base = row[base_colname]
        #rewrite if in recursion and counter is not original counter
        if counter != row['counter_ccy']:
            if counter == row['base_ccy']:
                usd_counter = row['rateUSDBASE']
            else:
                if counter == 'USD':
                    usd_counter = 1
                else:
                    counter_colname = 'rateUSD' + counter
                    usd_counter = row[counter_colname]
        return usd_counter / usd_base

    def get_base_trade_against_rate():
        #by default
        usd_base = row['rateUSDBASE']
        #rewrite if in recursion and base is not original base
        if base != row['base_ccy']:
            if base == row['counter_ccy']:
                usd_base = row['rateUSDCOUNTER']
            else:
                if base == 'USD':
                    usd_base = 1
                else:
                    base_colname = 'rateUSD' + base
                    usd_base = row[base_colname]
        if base_trades_against.upper() == 'USD':
            return 1/usd_base
        return row['rateUSD' + base_trades_against.upper()] / usd_base

    def get_counter_trade_against_rate():
        #by default
        usd_counter = row['rateUSDCOUNTER']
        #rewrite if in recursion and counter is not original counter
        if counter != row['counter_ccy']:
            if counter == row['base_ccy']:
                usd_counter = row['rateUSDBASE']
            else:
                if counter == 'USD':
                    usd_counter = 1
                else:
                    counter_colname = 'rateUSD' + counter
                    usd_counter = row[counter_colname]
        if counter_trades_against.upper() == 'USD':
            return 1/usd_counter
        return row['rateUSD' + counter_trades_against.upper()] / usd_counter

    def get_any_new_mark_rate(base, counter): 
        pairname = base + counter
        if pairname in new_markrate_dict.keys():
            return new_markrate_dict[pairname]
        usd_base = new_markrate_dict[('USD' + base)]
        usd_counter = new_markrate_dict[('USD' + counter)]
        new_markrate_dict[pairname] = 1/usd_base * usd_counter #better to add so no need to re-calculate when it's old_markrate (next iteration)
        return new_markrate_dict[pairname]

    def get_usd_base_mark_rate():
        return get_any_new_mark_rate('USD', base)

    def get_usd_counter_mark_rate():
        return get_any_new_mark_rate('USD', counter)

    def get_base_counter_mark_rate(): #this is the marking rate (some point after the trade)
        return get_any_new_mark_rate(base, counter)

    def get_counter_gbp_mark_rate():
        return get_any_new_mark_rate(counter, 'GBP')
     
    # Conventional pair for base currency
    try:
        base_conv_base = conventional_fx_pairs.at[base, 'base']
    except:
        print('Could not get rate for base currency ' + base + ' !')
        return
    base_conv_counter = conventional_fx_pairs.at[base, 'counter']
     
    # Conventional pair for counter currency
    try:
        counter_conv_base = conventional_fx_pairs.at[counter, 'base']
    except:
        print('Could not get rate for counter currency ' + counter + ' !')
        return
    counter_conv_counter = conventional_fx_pairs.at[counter, 'counter']
     
    # Check if the pair is already conventional
    if (base == base_conv_base and counter == base_conv_counter) or \
        (base == counter_conv_base and counter == counter_conv_counter):
        #Debug
        #print('Adding ' + str(amount) + ' to pair ' + pair + ', market rate moved to ' + str(get_base_counter_mark_rate()) + ' from ' + str(get_base_counter_rate()))
        # This will not happen as because if base and counter trades against the same, we don't recurse, and also because longest distance to USD thru one currecy (second is USD) and we terminate if pair is already conventional
        # if pair in row.keys():
        #     if ~row[pair].isnull():
        #         print('PAIR ALREADY ADDED TO THIS ROW')
        delta_in_counter = float(amount) * (get_base_counter_mark_rate() - get_base_counter_rate()) #calculating delta in counter ccy amount
        #Debug
        #print('Delta in ' + counter + ' ccy amount: ' + str(delta_in_counter))
        #print('Rate for ' + counter + 'GBP' + ' : ' + str(get_counter_gbp_mark_rate()))
        positions_dict[pair] = positions_dict.get(pair, 0) + float(amount)
        cum_deltas_dict[pair] = cum_deltas_dict.get(pair, 0) + delta_in_counter * get_counter_gbp_mark_rate() #delta on pair already + delta in GBP amount for current trade
        return
     
    # Check if the pair is inverted conventional meaning CounterBase is the conventional pair
    if (base == base_conv_counter and counter == base_conv_base) or \
        (base == counter_conv_counter and counter == counter_conv_base):
        # Invert currency pair, invert amount
        invpair = counter + base
        invamount = float(-amount) * get_base_counter_rate()
        #Debug
        #print('Adding ' + str(invamount) + ' to pair ' + invpair + ', market rate moved to ' + str(1/get_base_counter_mark_rate()) + ' from ' + str(1/get_base_counter_rate()))
        delta_in_base = float(invamount) * (1/get_base_counter_mark_rate() - 1/get_base_counter_rate()) #calculating delta in base ccy amount
        #Debug
        #print('Delta in ' + base + ' ccy amount: ' + str(delta_in_base))
        #print('Rate for ' + counter + 'GBP' + ' : ' + str(get_counter_gbp_mark_rate()))
        #print('Rate for ' + base + 'GBP' + ' : ' + str(get_base_counter_mark_rate() * get_counter_gbp_mark_rate()))
        positions_dict[invpair] = positions_dict.get(invpair, 0) + float(invamount)
        cum_deltas_dict[invpair] = cum_deltas_dict.get(invpair, 0) + delta_in_base * get_base_counter_mark_rate() * get_counter_gbp_mark_rate() #delta on pair already + delta in GBP amount for current trade
        return
     
    # Need to break down into components
    # At this stage result won't be a single pair
    # Can be 2, 3, or 4 pairs
     
    # 2 if base and counter trade against the same currency
    # 3 if base and counter trade against different currencies
    #     but one of them (trade against) is USD
    # 4 if base and counter trade against different currencies
    #     and both of them (trade against) are non-USD
     
    base_trades_against = conventional_fx_pairs.at[base, 'trades_against']
    counter_trades_against = conventional_fx_pairs.at[counter, 'trades_against']
          
    # This can be done without recursion with one more
    # if statement per pair
    calculate_delta(row, base, base_trades_against, amount)
    counter_amount = float(-amount) * get_base_counter_rate()
    calculate_delta(row, counter, counter_trades_against, counter_amount)
     
    trade_against_same = base_trades_against == counter_trades_against
     
    if not trade_against_same:
        # If we are here, means it's 3 or 4 pairs in result
        # "ta" stands here for trades_against
        # ta_trades_against is second iteration of "trades_against"
        base_ta_trades_against = \
            conventional_fx_pairs.at[base_trades_against, 'trades_against']
        counter_ta_trades_against = \
            conventional_fx_pairs.at[counter_trades_against, 'trades_against']
         
        # It is easier to follow with example
        # Say base ccy = PLN
        # Then base trades against = EUR
        # And base ta trades against = USD
        # We want to calculate amount in EUR for EURUSD trade
        # It is equal to PLN amount * PLN/EUR rate
        # Note the sign is same as amount
        # Same logic applies for couter currency

        base_ta_amount = float(amount) * get_base_trade_against_rate()
        counter_ta_amount = float(counter_amount) * get_counter_trade_against_rate()
         
        calculate_delta(row, base_trades_against, base_ta_trades_against, base_ta_amount)
        calculate_delta(row, counter_trades_against, counter_ta_trades_against, counter_ta_amount)
                  
    #Debug
    #print('Finished processing trade. Onto next...')
    #print()
    return

In [8]:
import duckdb
conn = duckdb.connect()

def run_query(prev_time, current_time):
    test = conn.execute(f'''
    
    with mark_rate as (select ccy2, unmarked_bid, unmarked_ask
    from 'Files/rates_2023_09_28_2.parquet'
    where ts = TIMESTAMP'{current_time}')
    
    SELECT t.*, rates_base.mid as rateUSDBASE, rates_counter.mid as rateUSDCOUNTER,
    rates_eur.mid as rateUSDEUR, rates_gbp.mid as rateUSDGBP, rates_zar.mid as rateUSDZAR, rates_inr.mid as rateUSDINR, rates_sgd.mid as rateUSDSGD, rates_hkd.mid as rateUSDHKD,
    mark_rate_base.mid as markrateUSDBASE, mark_rate_counter.mid as markrateUSDCOUNTER,
    
    FROM (SELECT date_trunc('second', es_stored_timestamp) as ts, base_ccy, counter_ccy, sum(base_amount*pow(10,-cd.exp)) as total_base_amount
    FROM 'Files/trades_2023_09_28.parquet' t
    JOIN (select ccy, exp from 'Files/ccy_details.parquet') cd
    ON t.base_ccy = cd.ccy
    where ts >= TIMESTAMP'{prev_time}' and ts < TIMESTAMP'{current_time}'
    group by ts, base_ccy, counter_ccy) t
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_base
    ON t.base_ccy = rates_base.ccy2 and t.ts = rates_base.ts
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_counter
    ON t.counter_ccy = rates_counter.ccy2 and t.ts = rates_counter.ts
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_eur
    ON t.ts = rates_eur.ts and rates_eur.ccy2 = 'EUR'
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_gbp
    ON t.ts = rates_gbp.ts and rates_gbp.ccy2 = 'GBP'
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_zar
    ON t.ts = rates_zar.ts and rates_zar.ccy2 = 'ZAR'
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_inr
    ON t.ts = rates_inr.ts and rates_inr.ccy2 = 'INR'
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_sgd
    ON t.ts = rates_sgd.ts and rates_sgd.ccy2 = 'SGD'
    JOIN (select ts, ccy2, (unmarked_bid+unmarked_ask)/2 as mid from 'Files/rates_2023_09_28_2.parquet') rates_hkd
    ON t.ts = rates_hkd.ts and rates_hkd.ccy2 = 'HKD'
    
    JOIN (select ccy2, (unmarked_bid+unmarked_ask)/2 as mid from mark_rate) mark_rate_base
    ON mark_rate_base.ccy2 = t.base_ccy
    JOIN (select ccy2, (unmarked_bid+unmarked_ask)/2 as mid from mark_rate) mark_rate_counter
    ON mark_rate_counter.ccy2 = t.counter_ccy
    
    ''').fetch_arrow_table().to_pandas()
    return test

def mark_query(current_time):
    test = conn.execute(f'''
    
    select ccy2, (unmarked_bid+unmarked_ask)/2 as mid
    from 'Files/rates_2023_09_28_2.parquet'
    where ts = TIMESTAMP'{current_time}'
    
    ''').fetch_arrow_table().to_pandas()
    return test

In [38]:
pair_maps = conventional_fx_pairs.copy()
pair_maps['pair'] = pair_maps['base'] + pair_maps['counter']
pair_to_counter = pair_maps.reset_index().set_index('pair')['counter'].to_dict()
pair_to_base = pair_maps.reset_index().set_index('pair')['base'].to_dict()

#will also be used below
def get_any_old_mark_rate(base, counter, old_markrate_dict): #if there was a pair in the current iteration that's not in the previos iteration, we still need to calc this because it won't be calculated when having processed the previous trades
        pairname = base + counter
        if pairname in old_markrate_dict.keys():
            return old_markrate_dict[pairname]
        usd_base = old_markrate_dict[('USD' + base)]
        usd_counter = old_markrate_dict[('USD' + counter)]
        old_markrate_dict[pairname] = 1/usd_base * usd_counter
        return old_markrate_dict[pairname]

def calculate_delta_on_existing_positions(existing_positions, old_markrate_dict, new_markrate_dict): #delta_on_existing_positions
    
    def get_any_new_mark_rate(base, counter): #if there was a pair in the previous iteration that's not in the current iteration, we still need to calc this because it won't be calculated when processing the new trades
        pairname = base + counter
        if pairname in new_markrate_dict.keys():
            return new_markrate_dict[pairname]
        usd_base = new_markrate_dict[('USD' + base)]
        usd_counter = new_markrate_dict[('USD' + counter)]
        new_markrate_dict[pairname] = 1/usd_base * usd_counter #better to add so no need to re-calculate when it's old_markrate (next iteration)
        return new_markrate_dict[pairname]
    
    delta_on_existing_positions = {}
    for key, value in existing_positions.items():
        counter = pair_to_counter[key]
        base = pair_to_base[key]
        delta = value * (get_any_new_mark_rate(base, counter) - get_any_old_mark_rate(base, counter, old_markrate_dict)) #delta on existing positions in counter currency
        delta_on_existing_positions[key] = delta * get_any_new_mark_rate(counter, 'GBP') #delta on existing positions in GBP
    return delta_on_existing_positions

In [70]:
def break_down_into_conventional(base, counter, amount, rates): 
    
    breakdown = pd.DataFrame(columns=['pair', 'amount'])
    pair = base + counter

    if base == counter:
        return breakdown
     
    base_conv_base = conventional_fx_pairs.at[base, 'base']
    base_conv_counter = conventional_fx_pairs.at[base, 'counter']
     
    counter_conv_base = conventional_fx_pairs.at[counter, 'base']
    counter_conv_counter = conventional_fx_pairs.at[counter, 'counter']
     
    if (base == base_conv_base and counter == base_conv_counter) or \
        (base == counter_conv_base and counter == counter_conv_counter):
        return pd.concat([breakdown if not breakdown.empty else None, pd.DataFrame(index = [0], data={'pair': pair, 'amount': amount})],
                                ignore_index=True)
     
    if (base == base_conv_counter and counter == base_conv_base) or \
        (base == counter_conv_counter and counter == counter_conv_base):
        # Invert currency pair, invert amount
        inv_pair = counter + base
        inv_amount = -amount * get_any_old_mark_rate(base, counter, rates)
        return pd.concat([breakdown if not breakdown.empty else None, pd.DataFrame(index = [0], data={'pair': inv_pair, 'amount': inv_amount})],
                                ignore_index=True)
     
    base_trades_against = conventional_fx_pairs.at[base, 'trades_against']
    counter_trades_against = conventional_fx_pairs.at[counter, 'trades_against']
     
    trade_against_same = base_trades_against == counter_trades_against
     
    base_conv = break_down_into_conventional(
        base, base_trades_against, amount, rates)
    counter_amount = -amount * get_any_old_mark_rate(base, counter, rates)
    counter_conv = break_down_into_conventional(
        counter, counter_trades_against, counter_amount, rates)
     
    breakdown = (breakdown.copy() if base_conv.empty else base_conv.copy() if breakdown.empty else pd.concat([breakdown, base_conv]))
    breakdown = (breakdown.copy() if counter_conv.empty else counter_conv.copy() if breakdown.empty else pd.concat([breakdown, counter_conv]))
     
    if not trade_against_same:
        base_ta_trades_against = \
            conventional_fx_pairs.at[base_trades_against, 'trades_against']
        counter_ta_trades_against = \
            conventional_fx_pairs.at[counter_trades_against, 'trades_against']
         
        base_ta_amount = amount * get_any_old_mark_rate(base, base_trades_against, rates)
        counter_ta_amount = counter_amount * get_any_old_mark_rate(counter, counter_trades_against, rates)
         
        base_ta_conv = break_down_into_conventional(
            base_trades_against, base_ta_trades_against,
            base_ta_amount, rates)
        counter_ta_conv = break_down_into_conventional(
            counter_trades_against, counter_ta_trades_against,
            counter_ta_amount, rates)
         
        breakdown = (breakdown.copy() if base_ta_conv.empty else base_ta_conv.copy() if breakdown.empty else pd.concat([breakdown, base_ta_conv]))
        breakdown = (breakdown.copy() if counter_ta_conv.empty else counter_ta_conv.copy() if breakdown.empty else pd.concat([breakdown, counter_ta_conv]))
         
    return breakdown

def balances_to_pairs(balances, old_markrate_dict):
     
    exposures = pd.DataFrame(columns=['pair', 'amount'])
     
    for currency in balances.index:
        amount = balances.at[currency, 'amount']
        breakdown = break_down_into_conventional(
            currency, 'GBP', amount, old_markrate_dict) #assuming everything traded for GBP (company currency), will be the same result (except for company base), no matter what we put here
        exposures = pd.concat([exposures  if not exposures.empty else None, breakdown if not breakdown.empty else None], ignore_index=True)
         
    exposures = exposures.groupby('pair').sum()
 
    return exposures

## Full day delta calculation with buffer for 28th September taking 27th September EOD positions

In [35]:
query = '''
select currency, sum(closing_amount_major) as amount
from trading.cob_fx_position_reporting
where cob_date = date'2023-09-27'
group by currency 
'''

single_ccy_pos = zeus_execution(query).set_index('currency')

[2023-12-04 15:17:48.111465] Connected successfully.


In [85]:
prev_time = '2023-09-28 00:00:00'
current_time = '2023-09-28 15:35:41'

cum_deltas_dict = dict()
old_markrate_dict = dict()
new_markrate_dict = dict()

marks = mark_query(prev_time)
marks['pair'] = 'USD' + marks['ccy2']
old_markrate_dict.update(marks.set_index('pair')['mid'].to_dict())

positions_dict = balances_to_pairs(single_ccy_pos, old_markrate_dict)['amount'].to_dict()
existing_positions = positions_dict.copy()

marks = mark_query(current_time)
marks['pair'] = 'USD' + marks['ccy2']
new_markrate_dict.update(marks.set_index('pair')['mid'].to_dict())

test = run_query(prev_time, current_time) #will update positions_dict etc.
_ = test.apply(calculate_delta, axis=1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [86]:
delta_on_existing_positions = calculate_delta_on_existing_positions(existing_positions, old_markrate_dict, new_markrate_dict)
total_delta = pd.Series(cum_deltas_dict).add(pd.Series(delta_on_existing_positions), fill_value=0.0)

In [88]:
total_delta.sort_values()

EURUSD    -87659.421271
USDKRW     -1643.720163
TRBUSD     -1377.172674
USDKZT     -1257.463167
EURMAD     -1199.280315
               ...     
ETHUSD      1561.558263
USDCOP      1563.680008
EURMDL      1821.655882
COMPUSD     2078.784311
GBPUSD     46701.212389
Length: 301, dtype: float64

## Delta calculation between 15:15 and 15:30 on 28th Sept and from 15:15 to 15:45 using delta and positions buffer

In [10]:
prev_time = '2023-09-28 15:15:00'
current_time = '2023-09-28 15:30:00'

cum_deltas_dict = dict()
positions_dict = dict()
old_markrate_dict = dict()
new_markrate_dict = dict()

marks = mark_query(current_time)
marks['pair'] = 'USD' + marks['ccy2']
new_markrate_dict.update(marks.set_index('pair')['mid'].to_dict())

test = run_query(prev_time, current_time)
_ = test.apply(calculate_delta, axis=1)

In [12]:
pd.Series(cum_deltas_dict).sort_values()

EURUSD    -87.428561
EURCHF    -21.944106
BTCUSD    -18.635034
XAUUSD    -13.095452
EURHUF    -12.825068
             ...    
USDTRY     12.333488
COMPUSD    16.907685
EURSEK     20.417848
USDBRL     34.285548
GBPUSD     51.576099
Length: 216, dtype: float64

In [ ]:
#Now 15 minutes later, we need to update the deltas, get the delta on new trades and update mark rates

prev_time = '2023-09-28 15:30:00'
current_time = '2023-09-28 15:45:00'
old_markrate_dict = new_markrate_dict
marks = mark_query(current_time)
marks['pair'] = 'USD' + marks['ccy2']
new_markrate_dict = marks.set_index('pair')['mid'].to_dict()
existing_positions = positions_dict.copy()
test2 = run_query(prev_time, current_time)
_ = test2.apply(calculate_delta, axis=1)

delta_on_existing_positions = calculate_delta_on_existing_positions(existing_positions, old_markrate_dict, new_markrate_dict)
total_delta = pd.Series(cum_deltas_dict).add(pd.Series(delta_on_existing_positions), fill_value=0.0)

In [36]:
total_delta.sort_values()

USDZAR    -77.536352
EURUSD    -73.550240
EURPLN    -43.419609
XAUUSD    -39.966910
USDTHB    -25.103667
             ...    
EURNOK     17.832802
AUDUSD     25.394488
USDBRL     35.842867
EURSEK     60.217922
GBPUSD    135.190665
Length: 231, dtype: float64

## Sanity check - delta should not depend on interval - checking 15:15 to 15:30 using 1 min intervals

In [42]:
from datetime import datetime
n_periods = 15
start_times = pd.date_range(start=datetime(2023,9,28,15,15), end=datetime(2023,9,28,15,29), periods=n_periods)
end_times = pd.date_range(start=datetime(2023,9,28,15,16), end=datetime(2023,9,28,15,30), periods=n_periods)

In [49]:
from tqdm import tqdm

cum_deltas_dict = dict()
positions_dict = dict() #positions are automatically accumulated
old_markrate_dict = dict()
new_markrate_dict = dict()

marks = mark_query(end_times[0])
marks['pair'] = 'USD' + marks['ccy2']
new_markrate_dict.update(marks.set_index('pair')['mid'].to_dict())

test = run_query(start_times[0], end_times[0])
_ = test.apply(calculate_delta, axis=1)

for i in tqdm(range(1,n_periods,1)):
    old_markrate_dict = new_markrate_dict #update old_markrate
    marks = mark_query(end_times[i])
    marks['pair'] = 'USD' + marks['ccy2']
    new_markrate_dict = marks.set_index('pair')['mid'].to_dict() #update new_markrate
    
    existing_positions = positions_dict.copy()
    test = run_query(start_times[i], end_times[i])
    _ = test.apply(calculate_delta, axis=1)
    delta_on_existing_positions = calculate_delta_on_existing_positions(existing_positions, old_markrate_dict, new_markrate_dict)
    cum_deltas_dict = pd.Series(cum_deltas_dict).add(pd.Series(delta_on_existing_positions), fill_value=0.0) #update cum_deltas

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  0%|          | 0/14 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  7%|▋         | 1/14 [00:06<01:23,  6.41s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

 86%|████████▌ | 12/14 [01:17<00:12,  6.45s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 93%|█████████▎| 13/14 [01:23<00:06,  6.44s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

100%|██████████| 14/14 [01:30<00:00,  6.45s/it]


In [50]:
cum_deltas_dict.sort_values() #basically the same

EURUSD    -87.409249
EURCHF    -21.939414
BTCUSD    -18.628250
XAUUSD    -13.096223
EURHUF    -12.824914
             ...    
USDTRY     12.333791
COMPUSD    16.905836
EURSEK     20.408776
USDBRL     34.246200
GBPUSD     51.563634
Length: 216, dtype: float64

## function unit testing

In [15]:
ut1 = pd.DataFrame([['PLN', 'MOP', -50.0, 4, 8, 0.5, 1, 8, 
                                          4, 8, 0.5, 0.5, 4], #everything increase by 1
                    ['GBP', 'RUB', -50.0, 0.5, 50, 0.5, 1, 8, 
                                          2.5, 52, 2.5, 3, 10]], #everything increase by 2
                    columns = ['base_ccy', 'counter_ccy', 'total_base_amount', 'rateUSDBASE', 'rateUSDCOUNTER', 'rateUSDGBP', 'rateUSDEUR', 'rateUSDHKD', 'markrateUSDBASE', 'markrateUSDCOUNTER', 'markrateUSDGBP', 'markrateUSDEUR', 'markrateUSDHKD'])

In [17]:
ut1.apply(calculate_delta, axis=1)

Adding 12.5 to pair EURPLN, market rate moved to 8.0 from 4.0
Delta in PLN ccy amount: 50.0
Rate for EURGBP : 1.0
Rate for PLNGBP : 0.125
Adding -100.0 to pair HKDMOP, market rate moved to 2.0 from 1.0
Delta in MOP ccy amount: -100.0
Rate for HKDGBP : 0.125
Rate for MOPGBP : 0.0625
Adding -12.5 to pair EURUSD, market rate moved to 2.0 from 1.0
Delta in USD ccy amount: -12.5
Rate for USDGBP : 0.5
Adding -12.5 to pair USDHKD, market rate moved to 4.0 from 8.0
Delta in HKD ccy amount: 50.0
Rate for USDGBP : 0.5
Rate for HKDGBP : 0.125
Finished processing trade. Onto next...

Adding -50.0 to pair GBPUSD, market rate moved to 0.4 from 2.0
Delta in USD ccy amount: 80.0
Rate for USDGBP : 2.5
Adding -100.0 to pair USDRUB, market rate moved to 52.0 from 50.0
Delta in RUB ccy amount: -200.0
Rate for USDGBP : 2.5
Rate for RUBGBP : 0.04807692307692308
Finished processing trade. Onto next...



,EURPLN,EURUSD,GBPUSD,HKDMOP,USDHKD,USDRUB,base_ccy,counter_ccy,markrateUSDBASE,markrateUSDCOUNTER,markrateUSDEUR,markrateUSDGBP,markrateUSDHKD,rateUSDBASE,rateUSDCOUNTER,rateUSDEUR,rateUSDGBP,rateUSDHKD,total_base_amount
0,6.25,-6.25,NaN,-6.25,6.25,NaN,PLN,MOP,4.0,8,0.5,0.5,4,4.0,8,1,0.5,8,-50.0
1,NaN,NaN,200.0,NaN,NaN,-9.615385,GBP,RUB,2.5,52,3.0,2.5,10,0.5,50,1,0.5,8,-50.0


In [ ]:
# conn.execute('''
# with mark_rate as (select ccy2, bid, ask
# from 'rates_2023_09_28_2.parquet'
# where ts = ?)

# select (bid+ask)/2 from mark_rate where ccy2 = 'EUR'
# ''',[mark_time]).df()

# conn.execute('''
# with mark_rate as (select ccy2, bid, ask
# from 'rates_2023_09_28_2.parquet'
# where ts = ?)

# select (bid+ask)/2 from mark_rate where ccy2 = 'EUR'
# ''',[mark_time]).df()

# orig = conn.execute('''
# with mark_rate as (select ccy2, bid, ask
# from 'rates_2023_09_28_2.parquet'
# where ts = ?)

# SELECT *,
# FROM (SELECT date_trunc('second', es_stored_timestamp) as ts, base_ccy, counter_ccy, sum(base_amount) as total_base_amount
# FROM 'trades_2023_09_28.parquet'
# where ts > TIMESTAMP '2023-09-28 15:30:00'
# group by ts, base_ccy, counter_ccy) t
# -- for some reason simple select didn't work
# CROSS JOIN (select (bid+ask)/2 as markrateUSDEUR from mark_rate where ccy2 = 'EUR')
# CROSS JOIN (select (bid+ask)/2 as markrateUSDGB from mark_rate P where ccy2 = 'GBP')

# ''',[mark_time]).df()

# def dummy(row):
#     row['+'] = row['num']+1
#     row['-'] = row['num']-1
#     if row['num']%3 == 0:
#         row['div3'] = True
#     return row

# test2 = pd.DataFrame(index=[0,1,2], columns=['num', 'random'], data = [[1, 0],[2, 10], [3, 29]])

# test2.apply(dummy, axis=1)

In [15]:
ts = '2023-09-28 15:45:32'
conn.execute('''
select *
from 'rates_2023_09_28_2.parquet'
where ts = ?
''', [ts]).df()

,ts,ccy2,bid,ask,unmarked_bid,unmarked_ask
0,2023-09-28 15:45:32,1INCH,3.696930,3.805188,3.745737,3.756244
1,2023-09-28 15:45:32,AAVE,0.015396,0.015799,0.015594,0.015601
2,2023-09-28 15:45:32,ACH,63.994084,66.000410,64.928403,65.064137
3,2023-09-28 15:45:32,ADA,3.974169,4.077592,4.024760,4.026973
4,2023-09-28 15:45:32,AED,3.666496,3.679324,3.672740,3.673080
...,...,...,...,...,...,...
324,2023-09-28 15:45:32,YGG,4.686115,4.770411,4.712268,4.744081
325,2023-09-28 15:45:32,ZAR,18.944012,19.046598,18.993300,18.997300
326,2023-09-28 15:45:32,ZMW,20.735371,20.814664,20.750000,20.800000
327,2023-09-28 15:45:32,ZRX,5.151368,5.310077,5.221016,5.240173


In [22]:
conventional_fx_pairs.trades_against.value_counts()

trades_against
USD    376
EUR     26
GBP      6
ZAR      3
INR      2
SGD      1
HKD      1
AUD      1
Name: count, dtype: int64

In [37]:
conventional_fx_pairs.loc['AUD']

trades_against                      USD
base                                AUD
counter                             USD
document_name     CONVENTIONAL_FX_PAIRS
sheet_name                       Sheet1
sheet_line_no                        17
Name: AUD, dtype: object

In [40]:
conventional_fx_pairs.loc[conventional_fx_pairs.trades_against=='EUR',:]

,trades_against,base,counter,document_name,sheet_name,sheet_line_no
currency,,,,,,
ALL,EUR,EUR,ALL,CONVENTIONAL_FX_PAIRS,Sheet1,8
BAM,EUR,EUR,BAM,CONVENTIONAL_FX_PAIRS,Sheet1,22
BGN,EUR,EUR,BGN,CONVENTIONAL_FX_PAIRS,Sheet1,28
CHF,EUR,EUR,CHF,CONVENTIONAL_FX_PAIRS,Sheet1,47
CVE,EUR,EUR,CVE,CONVENTIONAL_FX_PAIRS,Sheet1,60
CZK,EUR,EUR,CZK,CONVENTIONAL_FX_PAIRS,Sheet1,61
DKK,EUR,EUR,DKK,CONVENTIONAL_FX_PAIRS,Sheet1,63
HRK,EUR,EUR,HRK,CONVENTIONAL_FX_PAIRS,Sheet1,98
HUF,EUR,EUR,HUF,CONVENTIONAL_FX_PAIRS,Sheet1,100


In [3]:
conventional_fx_pairs.loc[conventional_fx_pairs.trades_against=='ZAR',:] #lsl pegged at par, nad is pegged at par, szl also pegged

,currency,trades_against,base,counter,document_name,sheet_name,sheet_line_no
134,LSL,ZAR,ZAR,LSL,CONVENTIONAL_FX_PAIRS,Sheet1,135
160,NAD,ZAR,ZAR,NAD,CONVENTIONAL_FX_PAIRS,Sheet1,161
218,SZL,ZAR,ZAR,SZL,CONVENTIONAL_FX_PAIRS,Sheet1,219


In [11]:
conventional_fx_pairs.loc[conventional_fx_pairs.trades_against=='INR',:] #BTN is basically pegged at par, INRNPR = 1.6 peg

,currency,trades_against,base,counter,document_name,sheet_name,sheet_line_no
37,BTN,INR,INR,BTN,CONVENTIONAL_FX_PAIRS,Sheet1,38
168,NPR,INR,INR,NPR,CONVENTIONAL_FX_PAIRS,Sheet1,169


In [5]:
conventional_fx_pairs.loc[conventional_fx_pairs.trades_against=='SGD',:] #pegged with SGD at par

,currency,trades_against,base,counter,document_name,sheet_name,sheet_line_no
31,BND,SGD,SGD,BND,CONVENTIONAL_FX_PAIRS,Sheet1,32


In [6]:
conventional_fx_pairs.loc[conventional_fx_pairs.trades_against=='HKD',:] #pegged to HKD at MOPHKD = 1.03

,currency,trades_against,base,counter,document_name,sheet_name,sheet_line_no
150,MOP,HKD,HKD,MOP,CONVENTIONAL_FX_PAIRS,Sheet1,151


In [7]:
conventional_fx_pairs.loc[conventional_fx_pairs.trades_against=='AUD',:] #pegged to AUD at parity

,currency,trades_against,base,counter,document_name,sheet_name,sheet_line_no
229,TVD,AUD,AUD,TVD,CONVENTIONAL_FX_PAIRS,Sheet1,230


In [23]:
conventional_fx_pairs.set_index('currency').loc[conventional_fx_pairs.trades_against.value_counts().keys(),:]

,trades_against,base,counter,document_name,sheet_name,sheet_line_no
trades_against,,,,,,
USD,USD,USD,USD,CONVENTIONAL_FX_PAIRS,Sheet1,237
EUR,USD,EUR,USD,CONVENTIONAL_FX_PAIRS,Sheet1,76
GBP,USD,GBP,USD,CONVENTIONAL_FX_PAIRS,Sheet1,83
ZAR,USD,USD,ZAR,CONVENTIONAL_FX_PAIRS,Sheet1,264
INR,USD,USD,INR,CONVENTIONAL_FX_PAIRS,Sheet1,106
SGD,USD,USD,SGD,CONVENTIONAL_FX_PAIRS,Sheet1,202
HKD,USD,USD,HKD,CONVENTIONAL_FX_PAIRS,Sheet1,96
AUD,USD,AUD,USD,CONVENTIONAL_FX_PAIRS,Sheet1,17


In [73]:
rates.loc[rates.ccy2 == 'TVD',:]

,ts,ccy2,bid,ask,unmarked_bid,unmarked_ask
